In [ ]:
import ollama
from sentence_transformers import SentenceTransformer
import numpy as np
from qdrant_client import QdrantClient
from qdrant_client.http import models
from faker import Faker

Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.
Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.


In [ ]:

embedding_model = SentenceTransformer('sentence-transformers/all-mpnet-base-v2')

# sample data
fake = Faker()
payload = [{"song": fake.sentence(), "artist": fake.name(), "year": fake.year()} for _ in range(1000)]

# embeddings
embeddings = []
for item in payload:
    text = f"{item['artist']} {item['song']}"
    embedding = embedding_model.encode(text)
    embeddings.append(embedding)

embeddings = np.array(embeddings)

# quadrant client
client = QdrantClient(host="localhost", port=6333)

my_collection = "song_collection"

# check if collection exists, if not, create it
try:
    client.get_collection(collection_name=my_collection)
    print(f"Collection '{my_collection}' already exists. Skipping creation.")
except Exception as e:
    print(f"Collection '{my_collection}' does not exist. Creating a new one.")
    client.create_collection(
        collection_name=my_collection,
        vectors_config=models.VectorParams(size=768, distance=models.Distance.COSINE)  
    )

# upsert data (embedding vectors and payload) into the Qdrant collection
index = list(range(len(payload)))
client.upsert(
    collection_name=my_collection,
    points=models.Batch(
        ids=index,
        vectors=embeddings.tolist(),
        payloads=payload
    )
)

# semantic search using a query
query = "What song was released in 1975?"
query_embedding = embedding_model.encode(query)

search_result = client.search(
    collection_name=my_collection,
    query_vector=query_embedding.tolist(),
    limit=10  # limit the results to top 10 similar items
)

print("Search Results from Qdrant:")
for hit in search_result:
    print(f"ID: {hit.id}, Score: {hit.score}, Payload: {hit.payload}")

# Mistral 7B via Ollama for a query-based response
response = ollama.chat(model="mistral-7b", messages=[{"role": "user", "content": query}])

print("\nResponse from Mistral 7B (via Ollama):")
print(response['text'])